# Music Generation with LSTM

In [23]:
import glob
import numpy as np
import pickle
from music21 import converter, instrument, note, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from keras_self_attention import SeqSelfAttention

In [24]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./full_set_beethoven_mozart directory. Call BEFORE train """
    notes = []
    durations = []
    files = "chopin/*.mid"

    for file in glob.glob(files):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi) #Change to only grab the piano???
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch) + " " +  str(element.quarterLength))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder) + " " + str(element.quarterLength))
            elif isinstance(element, note.Rest):
                notes.append(str(element.name)  + " " + str(element.quarterLength))

    pickle.dump(notes, open('data/notes.p', 'wb'))

    return notes

In [25]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = to_categorical(network_output)

    return (network_input, network_output)

In [26]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    units = 512
    dropout = 0.3
    
    model = Sequential()
    model.add(
        Bidirectional(
            LSTM(
                units,
                dropout=dropout,
                return_sequences=True
            ),
            input_shape=(network_input.shape[1], network_input.shape[2])
        )
    )
    model.add(Dropout(dropout))
    model.add(SeqSelfAttention(attention_activation='relu'))
    model.add(LSTM(units, dropout=dropout))
    model.add(Dropout(dropout))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(clipnorm=1.0))

    return model

In [31]:
def train_network(load_notes=False, to_load_model=False, learning_rate=None):
    """ Train a Neural Network to generate music """
    notes = None
    model = None

    if load_notes:
        notes = pickle.load(open('data/notes.p', 'rb'))
    else:
        notes = get_notes()
    
    n_vocab = len(set(notes))
    network_input, network_output = prepare_sequences(notes, n_vocab)

    if load_model:
        model = load_model('weights.hdf5', custom_objects={'SeqSelfAttention': SeqSelfAttention})
    else:
        model = create_network(network_input, n_vocab)

    if learning_rate != None:
        K.set_value(model.optimizer.learning_rate, learning_rate)

    checkpoint = ModelCheckpoint(
        'weights.hdf5',
        monitor='loss',
        save_best_only=True,
        mode='min'
    )

    callbacks_list = [checkpoint]
    
    model.summary()

    model.fit(
        x=network_input,
        y=network_output,
        batch_size=1024,
        epochs=2000,
        callbacks=callbacks_list
    )

In [ ]:
train_network(True, True, 0.0003)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 100, 1024)         2105344   
_________________________________________________________________
dropout (Dropout)            (None, 100, 1024)         0         
_________________________________________________________________
seq_self_attention (SeqSelfA (None, None, 1024)        65601     
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 2606)              1336878   
_________________________________________________________________
activation (Activation)      (None, 2606)              0